In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
# Set random seeds for reproducibility
torch.manual_seed(42)
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

Using device: cuda


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train = df.iloc[:, 1:].values
y_train = df.iloc[:, 0].values

In [5]:
df2 = pd.read_csv('/content/fashion-mnist_test.csv')

In [6]:
X_test=df2.iloc[:, 1:].values
y_test=df2.iloc[:, 0].values

In [7]:
X_train = X_train/255.0
X_test = X_test/255.0

In [13]:
class customDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32).reshape(-1,1,28,28)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]

In [14]:
train_dataset=customDataset(X_train,y_train)
test_dataset=customDataset(X_test,y_test)

In [22]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [19]:
class MyNN(nn.Module):
  def __init__(self,input_features):
    super().__init__()
    self.features=nn.Sequential(
        nn.Conv2d(input_features,32,kernel_size=3,padding="same"),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(32,64,kernel_size=3,padding="same"),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2,stride=2),
        #(1,28,28)->(32,28,28)->(32,14,14)->(64,14,14)->(64,7,7)
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(128,64),
        nn.ReLU(),

        nn.Linear(64,10)

    )

  def forward(self,x):
    x=self.features(x)
    x=self.classifier(x)
    return x



In [20]:
learning_rate = 0.01
epochs = 100
model = MyNN(1)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [23]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 0.49233515340884526
Epoch: 2 , Loss: 0.30205108025074007
Epoch: 3 , Loss: 0.25844186414976916
Epoch: 4 , Loss: 0.23171485992272695
Epoch: 5 , Loss: 0.21235854640603066
Epoch: 6 , Loss: 0.19268172342280546
Epoch: 7 , Loss: 0.1772073651726047
Epoch: 8 , Loss: 0.16625565831959246
Epoch: 9 , Loss: 0.151463181523482
Epoch: 10 , Loss: 0.14210286827584107
Epoch: 11 , Loss: 0.13061931320553025
Epoch: 12 , Loss: 0.12137059770822525
Epoch: 13 , Loss: 0.11529490308774014
Epoch: 14 , Loss: 0.105743476363644
Epoch: 15 , Loss: 0.09845015239703159
Epoch: 16 , Loss: 0.09170825344131639
Epoch: 17 , Loss: 0.08578668445286651
Epoch: 18 , Loss: 0.07886536775299659
Epoch: 19 , Loss: 0.07759709779260059
Epoch: 20 , Loss: 0.07138769583522031
Epoch: 21 , Loss: 0.06893649332703401
Epoch: 22 , Loss: 0.061194987384291986
Epoch: 23 , Loss: 0.05996077421850835
Epoch: 24 , Loss: 0.05515292038857782
Epoch: 25 , Loss: 0.05385745163781879
Epoch: 26 , Loss: 0.05302922565455083
Epoch: 27 , Loss: 0.05036

In [24]:
model.eval()

MyNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [25]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9293


In [26]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9999666666666667
